In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

(train_ds, train_labels), (test_ds, test_labels) = tfds.load(
"tf_flowers",split=["train[:70%]", "train[:30%]"],
batch_size=-1,
as_supervised=True,
)

train_ds = tf.image.resize(train_ds, (150, 150))
test_ds = tf.image.resize(test_ds, (150, 150))

train_labels = to_categorical(train_labels, num_classes=5)
test_labels = to_categorical(test_labels, num_classes=5)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout

model = Sequential()
model.add(Conv2D(filters=96, kernel_size=(11,11), strides=(4,4),
activation='relu', input_shape=train_ds[0].shape))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(5,5), strides=(1,1),
activation='relu', padding="same"))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1),
activation='relu', padding="same"))
model.add(Conv2D(filters=384, kernel_size=(3,3), strides=(1,1),
activation='relu', padding="same"))
model.add(Conv2D(filters=256, kernel_size=(3,3), strides=(1,1),
activation='relu', padding="same"))
model.add(MaxPooling2D(pool_size=(3,3), strides=(2,2)))
model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(5, activation='softmax'))

train_ds = train_ds / 255.0
test_ds = test_ds / 255.0
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'],)

es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)
history_alex = model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32,)

from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras import layers, models

base_model = VGG16(weights="imagenet", include_top=False,
input_shape=train_ds[0].shape)
base_model.trainable = False

train_ds = preprocess_input(train_ds)
test_ds = preprocess_input(test_ds)

base_model.summary()

flatten_layer = layers.Flatten()
dense_layer_1 = layers.Dense(50, activation='relu')
dense_layer_2 = layers.Dense(20, activation='relu')
prediction_layer = layers.Dense(5, activation='softmax')
model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer_1,
    dense_layer_2,
    prediction_layer
])

model.compile(
optimizer='adam',
loss='categorical_crossentropy',
metrics=['accuracy'],
)
es = EarlyStopping(monitor='val_accuracy', mode='max', patience=5, restore_best_weights=True)
history = model.fit(train_ds, train_labels, epochs=50, validation_split=0.2, batch_size=32,)

import matplotlib.pyplot as plt
plt.title("Model comparison over tf_flower dataset (Loss vs Epochs)")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.plot(history.history["val_loss"], label = "VGG16 Accuracy")
plt.plot(history_alex.history["val_loss"], label = "Trained AlexNet Accuracy")
plt.legend()
plt.show()
plt.title("Model comparison over tf_flower dataset (Accuracy vs Epochs)")
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.plot(history.history["val_accuracy"], label = "VGG16 Accuracy")
plt.plot(history_alex.history["val_accuracy"], label = "Trained AlexNet Accuracy")
plt.legend()
plt.show()